In [1]:
import pandas as pd

metasamples = pd.read_excel("../Metadata4.xlsx")

In [2]:
metasamples

,PATIENT_ID,SAMPLE_ID,SAMPLE_TYPE,GENDER,INCLUDED,DAYS_AFTER_TREATMENT,AGE_DX,VAR1,VAR2,LANE,FASTA1,FASTA2,ROUND,MSI_ISH,MSI_SCORE,TUMOR_ONLY
0,VHIO_01,1123_3_cfDNA,cfDNA,XX,Y,224.0,55,NaN,F,1,s3://scitron/exomes/1123_1.fastq.gz,s3://scitron/exomes/1123_2.fastq.gz,FIRST,MSS,8.32,No
1,VHIO_01,1261_2_cfDNA,cfDNA,XX,Y,310.0,55,NaN,F,1,s3://scitron/exomes/1261_1.fastq.gz,s3://scitron/exomes/1261_2.fastq.gz,FIRST,MSS,13.29,No
2,VHIO_01,B19_0970__VH_17_B_022863_A5__1_tumor,tumor,XX,Y,266.0,55,NaN,F,1,s3://scitron/exomes/B19_0970_S3_R1_001.fastq.gz,s3://scitron/exomes/B19_0970_S3_R2_001.fastq.gz,FIRST,MSS,1.95,No
3,VHIO_01,B19_0971__17_22863_A1__0_gDNA,gDNA,XX,Y,NaN,55,NaN,NaN,1,s3://scitron/exomes/B19_0971_S4_R1_001.fastq.gz,s3://scitron/exomes/B19_0971_S4_R2_001.fastq.gz,FIRST,NaN,Na,No
4,VHIO_02,1514_5_cfDNA,cfDNA,XY,Y,336.0,57,AFTER,F,1,s3://scitron/exomes/1514_1.fastq.gz,s3://scitron/exomes/1514_2.fastq.gz,FIRST,MSS,Na,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,VHIO_45,CTAX023_PDX,PDX,XX,Y,1461.0,46,NaN,E,1,s3://scitron/exomes/L16-6865_CGATGT_L002_R1_00...,s3://scitron/exomes/L16-6865_CGATGT_L002_R2_00...,FIRST,MSS,Na,Yes
138,VHIO_46,CTAX039_PDX,PDX,XY,N,1882.0,68,NaN,NaN,1,s3://scitron/exomes/L19-6482_CAATGG_L002_R1_00...,s3://scitron/exomes/L19-6482_CAATGG_L002_R2_00...,FIRST,NaN,Na,Yes
139,VHIO_47,CTAX002_PDX,PDX,XY,Y,223.0,60,NaN,E,1,s3://scitron/exomes/L16-6861_ACTTGA_L001_R1_00...,s3://scitron/exomes/L16-6861_ACTTGA_L001_R2_00...,FIRST,MSI,Na,Yes
140,VHIO_48,T074_PDX,PDX,XX,N,6.0,88,NaN,NaN,1,s3://scitron/exomes/L19-6485_TAGGAT_L001_R1_00...,s3://scitron/exomes/L19-6485_TAGGAT_L001_R2_00...,FIRST,NaN,Na,Yes


In [3]:
from pathlib import Path

PATH = "/Users/jfnavarro/Projects/Scitron/Results/VariantCalling"

# Add tumor only and MSI columns 
tumor_only_values = list()
msi_scores = list()
for index, row in metasamples.iterrows():
    sample_id = row['SAMPLE_ID']
    sample_patient = row['PATIENT_ID']
    tumor_only = not any(metasamples[metasamples['PATIENT_ID'] == sample_patient]['SAMPLE_TYPE'] == 'gDNA')
    germline = all(metasamples[metasamples['PATIENT_ID'] == sample_patient]['SAMPLE_TYPE'] == 'gDNA')
    if row['SAMPLE_TYPE'] == 'gDNA':
        tumor_only_values.append('Na')
    else:
        tumor_only_values.append("Yes" if tumor_only else 'No')
    msi_value = 'Na'
    if not tumor_only and not germline:
        gdna = metasamples[(metasamples['PATIENT_ID'] == sample_patient)
                           & (metasamples['SAMPLE_TYPE'] == 'gDNA')]['SAMPLE_ID'].to_numpy()[0]
        if row['SAMPLE_TYPE'] != 'gDNA':
            for elem in Path(PATH).rglob('*{}_vs_{}_msisensor'.format(sample_id, gdna)):
                with open(elem, 'r') as filehandler:
                    tokens = filehandler.readlines()[1]
                    _,_,msi_value = tokens.split("\t")
    msi_scores.append(msi_value.strip())
metasamples['MSI_SCORE'] = msi_scores
metasamples['TUMOR_ONLY'] = tumor_only_values

In [4]:
metasamples.to_excel('../Metadata4.xlsx', index=False)